### Size PV System

### PV System Sizing

In [1]:
# Function to count how many panels can fit in a provided rectangle.
from math import floor

def count_small_rectangles(l, w, sl, sw):
    if sl <= 0 or sw <= 0 or l <= 0 or w <= 0:
        raise ValueError("Dimensions must be positive numbers")

    count_lengthwise = (l // sl) * (w // sw)
    count_widthwise = (l // sw) * (w // sl)
    
    return floor(max(count_lengthwise, count_widthwise))

def count_small_rectangles(a, sl, sw):
    
    return floor(a / (sl * sw))

In [2]:
l = 40 # Example length for 1600 m2 area
w = 40 # Example width for 1600 m2 area
a = 1600 # Example area, m2
sl = 1.762 # PV Panel length
sw = 1.134 # PV Panel width
# n_panels = count_small_rectangles(l, w, sl, sw)

n_panels = count_small_rectangles(a, sl, sw)
print(f"Number of panels of shape {sl} x {sw} m that can fit in an area of {a} m^2: {n_panels:}")

Number of panels of shape 1.762 x 1.134 m that can fit in an area of 1600 m^2: 800


In [6]:
percentage_so = 0.4
percentage_ew = 0.6
n_panels_so = floor(n_panels * percentage_so)
n_panels_ew = floor(n_panels * percentage_ew)
n_panels_eo = n_panels_ew // 2
n_panels_wo = n_panels_ew // 2

assert n_panels == n_panels_so + n_panels_ew
assert n_panels_ew == n_panels_eo + n_panels_wo

print(f"From the total {n_panels} panels. There are {n_panels_so} panels facing south, {n_panels_eo} panels facing east and {n_panels_wo} panels facing west.")

From the total 800 panels. There are 320 panels facing south, 240 panels facing east and 240 panels facing west.


In [5]:
panel_nominal_power = 445 # Wp

panels_power = n_panels * panel_nominal_power
panels_power_so = n_panels_so * panel_nominal_power
panels_power_ew = n_panels_ew * panel_nominal_power
panels_power_eo = n_panels_eo * panel_nominal_power
panels_power_wo = n_panels_wo * panel_nominal_power

print(f"The total power of the panels is {panels_power / 1000: 0.2f} kWp.\nThe power of the panels facing south is {panels_power_so/ 1000: 0.2f} kWp\nThe power of the panels facing east is {panels_power_eo/ 1000: 0.2f} kWp and the power of the panels facing west is {panels_power_wo/ 1000: 0.2f} kWp.")

The total power of the panels is  356.00 kWp.
The power of the panels facing south is  142.40 kWp
The power of the panels facing east is  106.80 kWp and the power of the panels facing west is  106.80 kWp.


### PV production profile

The PV Production profiles were obtained from [this website](https://re.jrc.ec.europa.eu/pvg_tools/en/), using the peak capacity obtained above and also the following information:
* Germany Zip code 92224
* a slope of 30 degrees
* Azimuth of 0 for south, 90 for west and -90 for east facing panels

the csv resulting files are used below.

In [52]:
import pandas as pd

def postprocessing(df):
    df = df.iloc[:-7]
    df['time'] = pd.to_datetime(df['time'], format="%Y%m%d:%H%M")
    df = df.set_index('time')
    df = df.astype(float)
    return df

In [53]:
pv_south = postprocessing(pd.read_csv("data/pv_south_profile.csv", header=8))
pd_east = postprocessing(pd.read_csv("data/pv_east_profile.csv", header=8))
pd_west = postprocessing(pd.read_csv("data/pv_west_profile.csv", header=8))

In [54]:
pv_all = pv_south.copy(deep=True)

In [55]:
pv_all['P'] = pv_south["P"] + pd_east["P"] + pd_west["P"]
pv_all['G(i)'] = pv_south['G(i)'] + pd_east['G(i)'] + pd_west['G(i)']

In [60]:
pv_all.to_csv("data/pv_complete_profile.csv", index=True)